# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
M. Samland  ->  M. Samland  |  ['M. Samland']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
J. Schreiber  ->  J. Schreiber  |  ['J. Schreiber']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
Arxiv has 52 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.09041


extracting tarball to tmp_2307.09041...

 done.
Retrieving document from  https://arxiv.org/e-print/2307.09301


extracting tarball to tmp_2307.09301...

 done.


Found 94 bibliographic references in tmp_2307.09301/main.bbl.
syntax error in line 81: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.09301-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.09301) | **MINDS. Abundant water and varying C/O across the disk of Sz 98 as seen  by JWST/MIRI**  |
|| D. Gasman, et al. -- incl., <mark>M. Güdel</mark>, <mark>G. Perotti</mark>, <mark>M. Samland</mark>, <mark>J. Bouwman</mark>, <mark>S. Scheithauer</mark>, <mark>J. Schreiber</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2023-07-19*|
|*Comments*| *Submitted to A&A on May 25 2023. 18 pages, 11 figures*|
|**Abstract**| MIRI/MRS on board the JWST allows us to probe the inner regions of protoplanetary disks. Here we examine the disk around the classical T Tauri star Sz 98, which has an unusually large dust disk in the millimetre with a compact core. We focus on the H$_2$O emission through both its ro-vibrational and pure rotational emission. Furthermore, we compare our chemical findings with those obtained for the outer disk from Atacama Large Millimeter/submillimeter Array (ALMA) observations. In order to model the molecular features in the spectrum, the continuum was subtracted and LTE slab models were fitted. The spectrum was divided into different wavelength regions corresponding to H$_2$O lines of different excitation conditions, and the slab model fits were performed individually per region. We confidently detect CO, H$_2$O, OH, CO$_2$, and HCN in the emitting layers. The isotopologue H$^{18}_2$O is not detected. Additionally, no other organics, including C$_2$H$_2$, are detected. This indicates that the C/O ratio could be substantially below unity, in contrast with the outer disk. The H$_2$O emission traces a large radial disk surface region, as evidenced by the gradually changing excitation temperatures and emitting radii. The OH and CO$_2$ emission are relatively weak. It is likely that H$_2$O is not significantly photodissociated; either due to self-shielding against the stellar irradiation, or UV-shielding from small dust particles. The relative emitting strength of the different identified molecular features point towards UV-shielding of H$_2$O in the inner disk of Sz 98, with a thin layer of OH on top. The majority of the organic molecules are either hidden below the dust continuum, or not present. In general, the inferred composition points to a sub-solar C/O ratio (<0.5) in the inner disk, in contrast with the larger than unity C/O ratio in the gas in the outer disk found with ALMA. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.09041-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.09041) | **A candidate of binary black hole system in AGN with broad Balmer  emission lines having quite different line widths**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2023-07-19*|
|*Comments*| *14 pages, 1 table, 13 figures, Accepted to be published in MNRAS*|
|**Abstract**| In the manuscript, a candidate of sub-pc binary black hole (BBH) system is reported in SDSS J1257+2023 through different properties of broad Balmer emission lines. After subtractions of host galaxy contributions, Gaussian functions are applied to measure emission lines in SDSS J1257+2023, leading line width (second moment) 760${\rm km/s}$ of broad H$\beta$ to be 0.69 times of line width 1100${\rm km/s}$ of broad H$\alpha$, quite different from normal line width ratio 1.1 of broad H$\beta$ to broad H$\alpha$ in quasars. The quite broader component in broad H$\alpha$ in SDSS J1257+2023 can be confirmed with confidence level higher than $5\sigma$ through F-test technique, through different model functions applied to measure emission lines. The broad Balmer emission lines having different line widths can be naturally explained by a BBH system with different obscurations on central two independent BLRs. Meanwhile, through ZTF light curves and corresponding phase folded light curves well described by sinusoidal function, BBH system expected optical QPOs can be detected with periodicity about 1000days, confirmed with confidence level higher than $3\sigma$ by Generalized Lomb-Scargle periodogram. And through CAR process simulated light curves, confidence level higher than $2\sigma$ can be determined to support the optical QPOs in SDSS J1257+2023 not from intrinsic AGN activities, although the ZTF light curves have short time durations. Moreover, through oversimplified BBH system simulated results, studying different broad Balmer lines as signs of BBH systems in normal quasars with flux ratios around 4 of broad H$\alpha$ to broad H$\beta$ could be done in near future. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.09301.md
    + _build/html/tmp_2307.09301/./figures/water_opacity.png
    + _build/html/tmp_2307.09301/./figures/slab_total_all_noise.png
    + _build/html/tmp_2307.09301/./figures/chi2_region2_HCN.png
    + _build/html/tmp_2307.09301/./figures/chi2_region2_CO2.png
    + _build/html/tmp_2307.09301/./figures/chi2_region2_CO2_Q_only.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# MINDS. Abundant water and varying C/O across the disk of Sz 98 as seen by JWST/MIRI

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.09301-b31b1b.svg)](https://arxiv.org/abs/2307.09301)<mark>Appeared on: 2023-07-19</mark> -  _Submitted to A&A on May 25 2023. 18 pages, 11 figures_

</div>
<div id="authors">

D. Gasman, et al. -- incl., <mark>M. Güdel</mark>, <mark>G. Perotti</mark>, <mark>M. Samland</mark>, <mark>J. Bouwman</mark>, <mark>S. Scheithauer</mark>, <mark>J. Schreiber</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** The Mid-InfraRed Instrument (MIRI) Medium Resolution Spectrometer (MRS) on board the _James Webb_ Space Telescope ( _JWST_ ) allows us to probe the inner regions of protoplanetary disks, where the elevated temperatures result in an active chemistry and where the gas composition may dictate the composition of planets forming in this region. The disk around the classical T Tauri star Sz 98, which has an unusually large dust disk in the millimetre with a compact core, was observed with the MRS, and we examine its spectrum here. We aim to explain the observations and put the disk of Sz 98 in context with other disks, with a focus on the $\ce{H2O}$ emission through both its ro-vibrational and pure rotational emission. Furthermore, we compare our chemical findings with those obtained for the outer disk from Atacama Large Millimeter/submillimeter Array (ALMA) observations. In order to model the molecular features in the spectrum, the continuum was subtracted and local thermodynamic equilibrium (LTE) slab models were fitted. The spectrum was divided into different wavelength regions corresponding to $\ce{H2O}$ lines of different excitation conditions, and the slab model fits were performed individually per region. We confidently detect $\ce{CO}$ , $\ce{H2O}$ , $\ce{OH}$ , $\ce{CO2}$ , and $\ce{HCN}$ in the emitting layers. Despite the plethora of $\ce{H2O}$ -lines, the isotopologue H $_\text{2}^{\text{18}}$ O is not detected. Additionally, no other organics, including $\ce{C2H2}$ , are detected. This indicates that the C/O ratio could be substantially below unity, in contrast with the outer disk. The $\ce{H2O}$ emission traces a large radial disk surface region, as evidenced by the gradually changing excitation temperatures and emitting radii. Additionally, the $\ce{OH}$ and $\ce{CO2}$ emission are relatively weak. It is likely that $\ce{H2O}$ is not significantly photodissociated; either due to self-shielding against the stellar irradiation, or UV-shielding from small dust particles. While $\ce{H2O}$ is prominent and $\ce{OH}$ relatively weak, the line fluxes in the inner disk of Sz 98 are not outliers compared to other disks. The relative emitting strength of the different identified molecular features point towards UV-shielding of $\ce{H2O}$ in the inner disk of Sz 98, with a thin layer of $\ce{OH}$ on top. The majority of the organic molecules are either hidden below the dust continuum, or not present. In general, the inferred composition points to a sub-solar C/O ratio ( $<$ 0.5) in the inner disk, in contrast with the larger than unity C/O ratio in the gas in the outer disk found with ALMA.

</div>

<div id="div_fig1">

<img src="tmp_2307.09301/./figures/water_opacity.png" alt="Fig2" width="100%"/>

**Figure 2. -** Flux ratios of H$_\text{2}^{\text{16}}$O lines of the same upper level in the data compared to slab models (top); and H$_\text{2}^{\text{16}}$O/H$_\text{2}^{\text{18}}$O ratio in the data and slab models (bottom). The properties of the transitions can be found in Table \ref{tab:opacity}. The dotted black vertical line indicates the upper limit of the column density based on the $1\sigma$ level of the spectrum and the non-detection of the H$_\text{2}^{\text{18}}$O line. The column densities on the bottom panel are multiplied by 550 to show the H$_\text{2}^{\text{16}}$O column density. (*fig:water_opacity*)

</div>
<div id="div_fig2">

<img src="tmp_2307.09301/./figures/slab_total_all_noise.png" alt="Fig6" width="100%"/>

**Figure 6. -** Slab fits for the four different wavelength ranges. The top and bottom panels per region are the data and total model, and the individual slab models; respectively. Spurious spikes and features from the data reduction have been blanked out (see the list in App. \ref{app:blank}). The horizontal lines indicate the regions used to fit the molecules, or the region where $\sigma$ is estimated after subtracting the slab models. (*fig:spec_ranges*)

</div>
<div id="div_fig3">

<img src="tmp_2307.09301/./figures/chi2_region2_HCN.png" alt="Fig10.1" width="33%"/><img src="tmp_2307.09301/./figures/chi2_region2_CO2.png" alt="Fig10.2" width="33%"/><img src="tmp_2307.09301/./figures/chi2_region2_CO2_Q_only.png" alt="Fig10.3" width="33%"/>

**Figure 10. -** $\chi^2$ plots of \ce{HCN} and \ce{CO2} detected in region 2. The red, yellow, and green lines indicate the $1\sigma$, $2\sigma$, and $3\sigma$ confidence contours, respectively. The white contours show the emitting radii in astronomical units (0.1 to 2.0 au). The black cross corresponds to the best fit. The bottom plot shows the map for \ce{CO2} when fitting the $Q$-branch only. (*fig:chi2_region2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.09301"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

230  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
